# Educação e sucesso profissional.

A pipeline de dados construída para esse projeto foi inspirada no arquivo [Education & Career Success](https://www.kaggle.com/datasets/adilshamim8/education-and-career-success) extraído do Kaggle. O arquivo que inicialmente continha apenas uma tabela foi fragmentado em três tabelas distintas para uma melhor análise e compreensão dos dados. Quanto às transformações e formatações, esses permanecem iguais aos dados originais.

<img src="MVP_imagem_dataset.jpg" width="800">








####**Objetivo**: Explorar a relação entre o desempenho acadêmico e o sucesso profissional de estudantes que moram nos Estados Unidos (EUA). Ao final dessa análise é esperado que seja possível responder todas as perguntas localiazadas logo abaixo.

- Estudantes de universidades mais bem classificadas (university_ranking) ganham salários iniciais mais altos?
- Quais fatores mais contribuem para um crescimento rápido na carreira (years_to_promotion)?
- Estudantes de certas áreas (field_of_study) tem mais oportunidades de emprego do que outras?
- O número de estágios concluídos (interships_completed) tem impacto no salário inicial?
- Estudantes com pontuação alta em networking (networking_score) recebem mais ofertas de emprego?


#Descrição das colunas e dos dados numéricos e categóricos.

## Informações do estudante

- **Student_ID:** Identificador único para cada estudante;
- **Age:** Idade do estudante (18-30 anos);
- **Gender:** Gênero ( Masculino, Feminino ou Outro);

## Desempenho acadêmico

- **High_School_GPA:** Média escolar do ensino médio (escala de 2.0 - 4.0);
- **SAT_Score:** Pontuação no teste padronizado (900 - 1600);
- **University_Ranking:** Ranking da universidade frequentada (1-1000);
- **University_GPA:** Média acadêmica na universidade (escala de 2.0 - 4.0);
- **Field_of_Study:** Área de estudo (Ciência da Computação, Medicina, Administração/Negócios, Artes, Direito, Engenharia, Matemática);

## Habilidades e atividades extracurriculares

- **Internships_Completed:** – Número de estágios concluídos (0-4);
- **Projects_Completed:** – Número de projetos acadêmicos ou pessoais concluídos (0-9);
- **Certifications:** – Número de certificações adicionais obtidas (0-5);
- **Soft_Skills_Score:** – Avaliação das habilidades interpessoais (1-10);
- **Networking_Score:** Pontuação baseada em conexões e networking profissional (1-10);

## Resultados profissionais

- **Job_Offers:** Número de ofertas de emprego recebidas após a graduação (0-5);
- **Starting_Salary:** Salário inicial (anual) no primeiro emprego em dólares americanos (25.000 - 150.000);
- **Career_Satisfaction:** Nível de satisfação com a carreira (1-10);
- **Years_to_Promotion:** Tempo necessário para receber a primeira promoção (1-5 anos);
- **Current_Job_Level:** Nível de carreira atual (Júnior, Pleno, Sênior, Executivo);
- **Work_Life_Balance:** Avaliação do equilíbrio entre trabalho e vida pessoal (1-10);
- **Entrepreneurship:** Indica se a pessoa iniciou um negócio próprio (Sim/Não).






# Modelagem dos dados em Esquema Estrela

<img src="Esquema_Estrela_MVP._.jpg" width="800">

#Carregamento dos dados originais

###Usando o Spark e o Pandas para carregar todos os dados da tabela original diretamente do GitHub

In [0]:
%python
import pandas as pd
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("LoadGitHubCSV").getOrCreate()


github_url = "https://raw.githubusercontent.com/Davisdenner/mvp_engenharia_de_dados/refs/heads/main/Arquivos%20utilizados/education_career_success.csv"


df_pandas = pd.read_csv(github_url)

df_spark = spark.createDataFrame(df_pandas)

spark.sql("DROP TABLE IF EXISTS education_career_success")

df_spark.write.format("delta").mode("overwrite").saveAsTable("education_career_success")

print(" Dados carregados com sucesso no Databricks!")



###Visualizando a tabela original "education_carrer_success"

In [0]:
SELECT * FROM education_career_success;

<img src="Tabela _ education_carreer_success.png" width="1000">

##Fragmentando a tabela original em três tabelas distintas:
- education_info;
- experience_info;
- carrer_info.

##Catálogo das colunas utlizadas na tabela "education_info"

- **Student_ID**: Identificação única do estudante;
- **Age**: Idade do estudante (18-30);
- **Gender**: Gênero do estudante (Masculino, Feminino ou Outro);
- **High_School_GPA**: Média no ensino médio (escala de 2.0 - 4.0);
- **SAT_Score**: Pontuação no SAT (900 - 1600);
- **University_Ranking**: Ranking da universidade frequentada (1 - 1000);
- **University_GPA**: Média na universidade (escala de 2.0 - 4.0);
- **Field_of_Study**: Área de estudo (Ciência da Computação, Medicina, Administração/Negócios, Artes, Direito, Engenharia, Matemática).



In [0]:
CREATE TABLE IF NOT EXISTS education_info (
    Student_ID VARCHAR(50) PRIMARY KEY,
    Age INT,
    Gender VARCHAR(10),
    High_School_GPA FLOAT,
    SAT_Score INT,
    University_Ranking INT,
    University_GPA FLOAT,
    Field_of_Study VARCHAR(100)
);

   

In [0]:
INSERT INTO education_info (Student_ID, Age, Gender, High_School_GPA, SAT_Score, University_Ranking, University_GPA, Field_of_Study)
SELECT 
    Student_ID,
    Age,
    Gender,
    High_School_GPA,
    SAT_Score,
    University_Ranking,
    University_GPA,
    Field_of_Study
FROM education_career_success;


%md
###Visualizando a tabela "education_info"

In [0]:
SELECT*FROM education_info;

<img src="Tabela _ education_info.png" width="1000">

##Catálogo das colunas utlizadas na tabela "experience_info"

- **Student_ID**: Identificação única do estudante;
- **Internships_Completed**: Número de estágios concluídos (0-4);
- **Projects_Completed**: Número de projetos concluídos (0-9);
- **Certifications**: Número de certificações obtidas (0-5);
- **Soft_Skills_Score**: Avaliação de habilidades interpessoais (1-10);
- **Networking_Score**: Avaliação de networking (1-10).


In [0]:
CREATE TABLE IF NOT EXISTS experience_info (
    Student_ID VARCHAR(50),
    Internships_Completed INT,
    Projects_Completed INT,
    Certifications INT,
    Soft_Skills_Score INT,
    Networking_Score INT,
    CONSTRAINT fk_experience_student FOREIGN KEY (Student_ID) REFERENCES education_info(Student_ID)
);


In [0]:
INSERT INTO experience_info (Student_ID, Internships_Completed, Projects_Completed, Certifications, Soft_Skills_Score, Networking_Score)
SELECT 
    Student_ID,
    Internships_Completed,
    Projects_Completed,
    Certifications,
    Soft_Skills_Score,
    Networking_Score
FROM education_career_success;


###Visualizando a tabela "experience_info"

In [0]:
SELECT*FROM experience_info;

<img src="Tabela _ experience_info.png" width="1000">

###Catálogo das colunas utlizadas na tabela "career_info"

- **Student_ID**: Identificação única do estudante;
- **Job_Offers**: Número de ofertas de emprego recebidas após a graduação (0-5);
- **Starting_Salary**: Salário inicial (anual) no primeiro emprego em dólares americanos (25000 - 150000);
- **Career_Satisfaction**: Nível de satisfação com a carreira (escala de 1 a 10);
- **Years_to_Promotion**: Tempo necessário para receber a primeira promoção (1-5 anos);
- **Current_Job_Level**: Nível de carreira atual (Entry, Mid, Senior, Executive);
- **Work_Life_Balance**: Equilíbrio entre trabalho e vida pessoal (escala de 1 a 10);
- **Entrepreneurship**: Indica se a pessoa iniciou um negócio (Yes ou No).

In [0]:
CREATE TABLE IF NOT EXISTS career_info (
    Student_ID VARCHAR(50),
    Job_Offers INT,
    Starting_Salary FLOAT,
    Career_Satisfaction INT,
    Years_to_Promotion INT,
    Current_Job_Level VARCHAR(50),
    Work_Life_Balance INT,
    Entrepreneurship VARCHAR(50),
    CONSTRAINT fk_career_student FOREIGN KEY (Student_ID) REFERENCES education_info(Student_ID)
);


   

In [0]:
INSERT INTO career_info (Student_ID, Job_Offers, Starting_Salary, Career_Satisfaction, Years_to_Promotion, Current_Job_Level, Work_Life_Balance, Entrepreneurship)
SELECT 
    Student_ID,
    Job_Offers,
    Starting_Salary,
    Career_Satisfaction,
    Years_to_Promotion,
    Current_Job_Level,
    Work_Life_Balance,
    Entrepreneurship
FROM education_career_success;


###Visualizando a tabela "carrer_info"

In [0]:
SELECT*FROM career_info;

<img src="Tabela _ career_info.png" width="1000">